In [ ]:
# import the necessary packages
import numpy as np
import argparse
import imutils
import glob
import cv2
from sklearn.neighbors import KNeighborsClassifier
from skimage import exposure
from skimage import feature
from imutils import paths


In [ ]:
data = []
labels = []

In [ ]:
for imagePath in paths.list_images("../train"):
    # extract the make of the car
    make = imagePath.split("/")[-2]
 
    # load the image, convert it to grayscale, and detect edges
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (200, 200))
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edged = imutils.auto_canny(gray)
 

    cnts, _ = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL,
        cv2.CHAIN_APPROX_SIMPLE)
    #cnts = cnts[0] if imutils.is_cv2() else cnts[1]
    c = max(cnts, key=cv2.contourArea)

    (x, y, w, h) = cv2.boundingRect(edged) # "c" instead of "edged" to exract only logo from picture
    logo = gray[y:y + h, x:x + w]
    logo = cv2.resize(logo, (200, 100))

    
    H = feature.hog(logo, orientations=9, pixels_per_cell=(10, 10),
        cells_per_block=(2, 2), transform_sqrt=True, block_norm="L1")
    # update the data and labels
    data.append(H)
    labels.append(make)


In [ ]:
# "train" the nearest neighbors classifier
print("training classifier...")
model = KNeighborsClassifier(n_neighbors=1)
model.fit(data, labels)
print("evaluating...")

[INFO] training classifier...
[INFO] evaluating...


In [ ]:
# loop over the test dataset
for (i, imagePath) in enumerate(paths.list_images("../testdata")):
   
    image = cv2.imread(imagePath)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    logo = cv2.resize(gray, (200, 100))
 
   
    (H, hogImage) = feature.hog(logo, orientations=9, pixels_per_cell=(10, 10),
        cells_per_block=(2, 2), transform_sqrt=True, block_norm="L1", visualize=True)
    pred = model.predict(H.reshape(1, -1))[0]
 
    hogImage = exposure.rescale_intensity(hogImage, out_range=(0, 255))
    hogImage = hogImage.astype("uint8")
    cv2.imshow("HOG Image #{}".format(i + 1), hogImage)
     
    image = cv2.resize(image, (400, 400))
    # prediction on the test image and display it
    cv2.putText(image, pred.title(), (10, 35), cv2.FONT_HERSHEY_SIMPLEX, 1.0,
        (0, 255, 0), 3)
    cv2.imshow("Test Image #{}".format(i + 1), image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()